# With ChatModel

In [115]:
import json

from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from langchain.schema import StrOutputParser

In [116]:
model = ChatOpenAI(
    model='gpt-4',
    api_key=open('api.txt', 'r').read(),
    temperature=0
)

In [117]:
with open('./data/json/last_row.json') as f:
    json_sample = json.load(f)

for JSOn Pitput ? https://www.youtube.com/watch?v=4vjYkKnGmFs

In [118]:
string_parser = model | StrOutputParser()

In [119]:
init_prompt = load_prompt('prompts/init/init_prompt_V01.json')
sumLen_prompt = load_prompt('prompts/summary/sumLen_prompt_V01.json').format(min= 6, max= 10)

In [120]:
chain = init_prompt | model | StrOutputParser()
print(init_prompt.format(best_practice= sumLen_prompt, ticket=json_sample))

 ### Instruction ###
Act as a professional Software Engineer working with issue trackers like Jira or Azure DevOps.
Working with these issue trackers, you have to consider multiple text-based best practices to increase understandability and reduce time.
Revise a ticket for compliance with these best practices and offer recommendations if needed.
The best practice to ckeck is: 

Best Practice: Summary Length
Your Task is to check if the summary of the ticket is between 6 to 10 words.
If the word count is in this range return the original summary.
Else, recommend a new summary fot the ticket.
Please return only the summary string and nothing else.


### Context ###
A ticket consist of multiple fields. You are provided with the fields and a short desciption for each field.
Assignee: The person responsible to resolve the issue.
Comments: Community discussion on the issue.
Components: Project components to which the issue belongs.
CreatedDate: The time and date the issue was created.
Creato

In [121]:
result = chain.invoke({"best_practice": sumLen_prompt,"ticket": json_sample})
result

"'Sourcetree crashes on opening'"